# WEB_SCRAPING_ASSIGNMENT_3

1. Write a python program which searches all the product under a particular product from www.amazon.in. 
The product to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search 
for guitars

In [1]:
# Importing libraries
import pandas as pd
import selenium
import time
from bs4 import BeautifulSoup 
from selenium import webdriver
from selenium.webdriver.common.by import By

from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException
import requests
import re

In [2]:
# Importing warings and ignoring waring if any present
import warnings
warnings.filterwarnings('ignore')

In [19]:
# connect to webdriver
driver = webdriver.Chrome("chromedriver.exe")

# get webpage
driver.get("https://www.amazon.in/")
driver.maximize_window()
time.sleep(2)

In [20]:
# search the user input product
search_bar = driver.find_element(By.ID,"twotabsearchtextbox")    
search_bar.clear()                                               
search_bar.send_keys(input('Enter the product you want to search : '))
time.sleep(3)
driver.find_element(By.XPATH,'//div[@class="nav-search-submit nav-sprite"]/span/input').click()

Enter the product you want to search : laptop


2. In the above question, now scrape the following details of each product listed in first 3 pages of your 
search results and save it in a data frame and csv. In case if any product has less than 3 pages in search 
results then scrape all the products available under that product name. Details to be scraped are: "Brand 
Name", "Name of the Product", "Price", "Return/Exchange", "Expected Delivery", "Availability" and 
“Product URL”. In case, if any of the details are missing for any of the product then replace it by “-“.

In [21]:
b_name=[]#Brand name
product_name=[]#Product Name
price=[]#Price
ret_exchange=[]#return/Exchange
exp_delivery=[]#Expected delivery
availability=[]#Availability
product_url=[]
urls=[]
for i in range(0,3):
    if i==0:
        p_urls=driver.find_elements(By.XPATH,"//h2[@class='a-size-mini a-spacing-none a-color-base s-line-clamp-2']/a")
    if i==1:
        n_button=driver.find_element(By.XPATH,'/html/body/div[1]/div[2]/div[1]/div[1]/div/span[3]/div[2]/div[29]/div/div/span/a[3]')
        n_button.click()
        time.sleep(2)
        p_urls=driver.find_elements(By.XPATH,"//h2[@class='a-size-mini a-spacing-none a-color-base s-line-clamp-2']/a")
   
    # selecting and clicking next page
    if i==2:
        n_button=driver.find_element(By.XPATH,'/html/body/div[1]/div[2]/div[1]/div[1]/div/span[3]/div[2]/div[27]/div/div/span/a[4]')
        n_button.click()
        time.sleep(2)
        p_urls=driver.find_elements(By.XPATH,"//h2[@class='a-size-mini a-spacing-none a-color-base s-line-clamp-2']/a")
            
            

    for i in p_urls:
        urls.append(i.get_attribute('href'))

    
for i in urls:
    driver.get(i)
    time.sleep(2)
#scraping data for brand name
    try:
        bran=driver.find_element(By.XPATH,"//a[@id='bylineInfo']")
        b_name.append(bran.text)
    except NoSuchElementException as e:
        b_name.append("--")

#scraping data for product name
    try:
        prod=driver.find_element(By.XPATH,"//h1[@id='title']/span")
        product_name.append(prod.text)
    except NoSuchElementException as e:
        product_name.append("--")


#scraping data for price
    try:
        pri=driver.find_element(By.XPATH,"//div[@id='apex_desktop']/div/div/table/tbody/tr[2]/td[2]/span/span[2]")
        price.append(pri.text)
    except NoSuchElementException as e:
            price.append("--")

#scaping data for return
    try:
        retu=driver.find_element(By.XPATH,"//div[@id='RETURNS_POLICY']/span/div[2]/a")
        ret_exchange.append(retu.text)
    except NoSuchElementException as e:
        ret_exchange.append("--")

#scraping data for expected delivery
    try:
        deliv=driver.find_element(By.XPATH,"//div[@id='ddmDeliveryMessage']/b")
        exp_delivery.append(deliv.text)
    except NoSuchElementException as e:
        exp_delivery.append("--")

#scraping data for availability
    try:
        avail=driver.find_element(By.XPATH,"//div[@id='availability']/span")
        availability.append(avail.text)
    except NoSuchElementException as e:
        availability.append("--")
              

In [22]:
print(len(availability),len(b_name),len(product_name),len(ret_exchange),len(exp_delivery),len(availability),len(price),len(urls))

69 69 69 69 69 69 69 69


In [23]:
#All the data into dataframe
df=pd.DataFrame({"Brand Name":b_name,"Product":product_name,"Price":price,"Return/Exchange":ret_exchange,"Expected Delivery":exp_delivery,"Availability":availability})#"Product Url":urls})
df

,Brand Name,Product,Price,Return/Exchange,Expected Delivery,Availability
0,Visit the HP Store,HP Pavilion 14 12th Gen Intel Core i5 16GB SDR...,--,7 Days Replacement,--,In stock.
1,Visit the Acer Store,Acer Aspire 3 Laptop Intel core i3 12th Gen - ...,--,7 Days Replacement,--,In stock.
2,Visit the Lenovo Store,Lenovo IdeaPad Slim 3 Intel Celeron N4020 15.6...,--,7 Days Replacement,--,In stock.
3,Visit the ASUS Store,"ASUS VivoBook 15 (2021), 15.6-inch (39.62 cm) ...",--,7 Days Replacement,--,In stock.
4,Visit the Lenovo Store,Lenovo IdeaPad Slim 1 Intel Celeron N4020 11.6...,--,7 Days Replacement,--,In stock.
...,...,...,...,...,...,...
64,Visit the Acer Store,Acer One 14 Business Laptop AMD Ryzen 3 3250U ...,--,7 Days Replacement,--,In stock.
65,Visit the HP Store,"HP 14s, 11th Gen Intel Core i3-1115G4, 8GB RAM...",--,7 Days Replacement,--,In stock.
66,Visit the Acer Store,Acer Extensa 15 Lightweight Laptop 11th Gen In...,--,7 Days Replacement,--,In stock.
67,Visit the Acer Store,Acer Extensa 15 Lightweight Laptop 11th Gen In...,--,7 Days Replacement,--,In stock.


In [24]:
df.to_csv("Scraped Data",index=False)

In [25]:
driver.close()

3. Write a python program to access the search bar and search button on images.google.com and scrape 10
images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’, ‘Guitar’, ‘Cakes’.

In [14]:
driver = webdriver.Chrome("chromedriver.exe")
driver.maximize_window()
driver.get('https://images.google.com/')

inp = ['fruits','cars','Machine Learning','Guitar','Cakes'] # input list

#Creating Empty lists
fruits=[]
cars=[]
ML=[]
Guitar=[]
Cakes=[]

lists = [] # list of empty lists

for i in inp:
    try:
        search_bar=driver.find_element(By.XPATH,'/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input') # Finding the search bar using it's xpath
    except NoSuchElementException:
        search_bar=driver.find_element(By.XPATH,'//input[@class="og3lId"]') # Finding the search bar using it's xpath
        
    search_bar.clear()
    search_bar.send_keys(i) # Inputing input keyword to search rock images
    
    try:
        button=driver.find_element(By.XPATH,'/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/button')  
    except NoSuchElementException:
        button=driver.find_element(By.XPATH,'//button[@class="rCGXm"]')
    button.click()
    
    
    img = driver.find_elements(By.XPATH,'//div[@class="bRMDJf islir"]/img')
    for k in img:
        try:
            source= k.get_attribute('src')
            if source is not None:
                if (source[0:4]=="http"):
                    lists.append(source)
        except NoSuchElementException:
            lists.append('-')
            

len(lists)            
#print(len(fruits),len(cars),len(ML),len(Guitar),len(Cakes))

139

In [15]:
fruits=lists[1:21]
cars=lists[22:42]
ML=lists[43:63]
Guitar=lists[64:84]
Cakes=lists[85:105]

print(len(fruits),len(cars),len(ML),len(Guitar),len(Cakes))

20 20 20 20 20


In [17]:
#creating a dataframe
df=pd.DataFrame({'Fruits':fruits,
                 'Cars':cars,
                 'Machine Learning':ML,
                 'Guitar':Guitar,
                 'Cakes':Cakes})

#printing dataframe
df.head(10)

,Fruits,Cars,Machine Learning,Guitar,Cakes
0,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://encrypted-tbn0.gstatic.com/images?q=tb...
1,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://encrypted-tbn0.gstatic.com/images?q=tb...
2,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://encrypted-tbn0.gstatic.com/images?q=tb...
3,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://encrypted-tbn0.gstatic.com/images?q=tb...
4,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://encrypted-tbn0.gstatic.com/images?q=tb...
5,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://encrypted-tbn0.gstatic.com/images?q=tb...
6,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://encrypted-tbn0.gstatic.com/images?q=tb...
7,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://encrypted-tbn0.gstatic.com/images?q=tb...
8,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://encrypted-tbn0.gstatic.com/images?q=tb...
9,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://encrypted-tbn0.gstatic.com/images?q=tb...


In [18]:
driver.close()

4. Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on
www.flipkart.com and scrape following details for all the search results displayed on 1st page. Details to be 
scraped: “Brand Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”, 
“Secondary Camera”, “Display Size”, “Battery Capacity”, “Price”, “Product URL”. Incase if any of the 
details is missing then replace it by “- “. Save your results in a dataframe and CSV.

In [26]:
# Connecting to webdriver
driver = webdriver.Chrome("chromedriver.exe")
driver.maximize_window()

# opening the url
driver.get('https://www.flipkart.com')

#cancelling login window
search_btn = driver.find_element(By.XPATH,'//button[@class="_2KpZ6l _2doB4z"]')
search_btn.click()

# getting the input
search_inp = input("Enter the product: ")

# locating the search bar
search_bar = driver.find_element(By.XPATH,"//input[@class='_3704LK']")
search_bar.send_keys(search_inp)

# locating the search button
search_btn = driver.find_element(By.XPATH,'//button[@class="L0Z3Pu"]')
search_btn.click()

Enter the product: pixel 4A


In [27]:
# creating Empty variable to store our data:
brand = []
s_name = []
color = []
ram = []
rom = []
p_camera = []
s_camera = []
display_s = []
display_r = []
battery = []
price = []
url = []

In [28]:
# fetching the details

# extracting product name, brand name, colour.
for i in driver.find_elements(By.XPATH,"//div[@class='_4rR01T']"):
    s_name.append(i.text)
    name = s_name[-1].split() #extracting the brand name from the product name
    brand.append(name[0])
    a = s_name[-1][s_name[-1].find("(")+1:s_name[-1].rfind(")")]
    color.append(a.split(',')[0]) #extracting colour from the product name
    
# extracting the ram and rom.
for j in driver.find_elements(By.XPATH,"//ul[@class='_1xgFaf']/li[1]"):
    ram.append(j.text.split('|')[0])
    rom.append(j.text.split('|')[1])
    
# extracting primary and secondary camera
for k in driver.find_elements(By.XPATH,"//ul[@class='_1xgFaf']/li[3]"):
    if(len(k.text.split('|')) == 2):
        p_camera.append(k.text.split('|')[0])
        s_camera.append(k.text.split('|')[1])
    else:
        p_camera.append(k.text.split('|')[0])
        s_camera.append('--')

# extracting display size and display resolution
for l in driver.find_elements(By.XPATH,"//ul[@class='_1xgFaf']/li[2]"):
    s = l.text[l.text.find("(")+1:l.text.rfind(")")]
    display_s.append(s)
    r = l.text[l.text.find(")")+1:]
    display_r.append(r)
        
#extracting battery power.
for n in driver.find_elements(By.XPATH,"//ul[@class='_1xgFaf']/li[4]"):
    battery.append(n.text)
    
# extracting price
for o in driver.find_elements(By.XPATH,"//div[@class='_30jeq3 _1_WHN1']"):
    price.append(o.text)

# extracting url
for p in driver.find_elements(By.XPATH,"//a[@class='_1fQZEK']"):
    url.append(p.get_attribute('href'))

In [29]:
flipkart = pd.DataFrame({})
flipkart['Brand'] = brand
flipkart['Product description'] = s_name
flipkart['Colour'] = color
flipkart['RAM'] = ram
flipkart['ROM'] = rom
flipkart['Primary Camera'] = p_camera
flipkart['Secondary Camera'] = s_camera
flipkart['Display Size'] = display_s
flipkart['Display Resolution'] = display_r
flipkart['Battery'] = battery
flipkart['Price'] = price
flipkart['Product url'] = url


In [30]:
flipkart.head()

,Brand,Product description,Colour,RAM,ROM,Primary Camera,Secondary Camera,Display Size,Display Resolution,Battery,Price,Product url
0,Google,"Google Pixel 4a (Just Black, 128 GB)",Just Black,6 GB RAM,128 GB ROM,12.2MP Rear Camera,8MP Front Camera,5.81 inch,Full HD+ Display,3140 mAh Battery,"₹31,999",https://www.flipkart.com/google-pixel-4a-just-...
1,Realme,Realme C30 - Locked with Airtel Prepaid,Realme C30 - Locked with Airtel Prepai,2 GB RAM,32 GB ROM,8MP Rear Camera,5MP Front Camera,6.5 inch,HD+ Display,5000 mAh Lithium Ion Battery,"₹6,899",https://www.flipkart.com/realme-c30-locked-air...
2,Realme,Realme C30 - Locked with Airtel Prepaid,Realme C30 - Locked with Airtel Prepai,2 GB RAM,32 GB ROM,8MP Rear Camera,5MP Front Camera,6.5 inch,HD+ Display,5000 mAh Lithium Ion Battery,"₹6,899",https://www.flipkart.com/realme-c30-locked-air...
3,Realme,Realme C30 - Locked with Airtel Prepaid,Realme C30 - Locked with Airtel Prepai,2 GB RAM,32 GB ROM,8MP Rear Camera,5MP Front Camera,6.5 inch,HD+ Display,5000 mAh Lithium Ion Battery,"₹6,899",https://www.flipkart.com/realme-c30-locked-air...
4,vivo,"vivo T1 44W (Starry Sky, 128 GB)",Starry Sky,4 GB RAM,128 GB ROM,50MP + 2MP + 2MP,16MP Front Camera,6.44 inch,Full HD+ AMOLED Display,5000 mAh Lithium Battery,"₹14,499",https://www.flipkart.com/vivo-t1-44w-starry-sk...


In [31]:
flipkart.to_csv("Scraped mobile File",index=False)

In [32]:
driver.close()

5. Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google
maps.

In [33]:
# Importing libraries
import pandas as pd
import selenium
import time
from bs4 import BeautifulSoup 
from selenium import webdriver
from selenium.webdriver.common.by import By

from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException
import requests
import re

# Importing warings
import warnings
warnings.filterwarnings('ignore')

In [34]:
driver = webdriver.Chrome("chromedriver.exe")
driver.maximize_window()

# opening the url
driver.get('https://www.google.com/maps')

search_inp = input("Enter the city: ")
search_bar = driver.find_element(By.XPATH,"//input[@id='searchboxinput']") # locating search bar
search_bar.send_keys(search_inp)

# locating search button
search_btn = driver.find_element(By.XPATH,"//button[@id='searchbox-searchbutton']")
search_btn.click()

# getting the geospatial co-ordinates from the url
time.sleep(2)
geo = driver.current_url
r = geo[geo.find("@")+1:]
s = r[0:21]
s = s.split(',')
print(f'''Latitude: {s[0]}
Longitude: {s[1]}''')

Enter the city: Vadodara
Latitude: 22.3220876
Longitude: 73.1030456


In [37]:
driver.close()

6. Write a program to scrap details of all the funding deals for second quarter (i.e Jan 21 – March 21) 
from trak.in.

In [38]:
driver = webdriver.Chrome("chromedriver.exe")

# get webpage
driver.get("https://trak.in/")
time.sleep(2)
driver.maximize_window()

In [39]:
#click on funding deals and go to that page
fun_btn=driver.find_element(By.XPATH,"//li[@id='menu-item-51510']/a").get_attribute('href')
driver.get(fun_btn)

In [40]:
# Creating empty list
funding={}
funding["Date"]=[]
funding["Startup Name"]=[]
funding["Industry or Vertical"]=[]
funding["Sub Vertical"]=[]
funding["Location"]=[]
funding["Investor"]=[]
funding["Investment Type"]=[]
funding["Amount"]=[]

for i in range(54,57):


    # Scraping data of Date
    date_tags = driver.find_elements(By.XPATH,'//table[@id="tablepress-{}"]/tbody/tr/td[2]'.format(i))# {} and .format () used
    #so the number coming in function format(n) will replace {}
    for date in date_tags:
        funding['Date'].append(date.text)

    # Scraping data of Startup Name
    name_tags = driver.find_elements(By.XPATH,'//table[@id="tablepress-{}"]/tbody/tr/td[3]'.format(i))
    for name in name_tags:
        funding['Startup Name'].append(name.text)
    
    # Scraping data of Industry OR Vertical
    ind_tags = driver.find_elements(By.XPATH,'//table[@id="tablepress-{}"]/tbody/tr/td[4]'.format(i))
    for n in ind_tags:
        funding['Industry or Vertical'].append(n.text)
    
    # Scraping data of Sub-Vertical
    sv_tags = driver.find_elements(By.XPATH,'//table[@id="tablepress-{}"]/tbody/tr/td[5]'.format(i))
    for sv in sv_tags:
        funding['Sub Vertical'].append(sv.text)

    # Scraping data of Location
    loc_tags = driver.find_elements(By.XPATH,'//table[@id="tablepress-{}"]/tbody/tr/td[6]'.format(i))
    for loc in loc_tags:
        funding['Location'].append(loc.text)
    
    # Scraping data of Investor
    inv_tags = driver.find_elements(By.XPATH,'//table[@id="tablepress-{}"]/tbody/tr/td[7]'.format(i))
    for inv in inv_tags:
        funding['Investor'].append(inv.text)
        
    # Scraping data of Investment Type
    invt_tags = driver.find_elements(By.XPATH,'//table[@id="tablepress-{}"]/tbody/tr/td[8]'.format(i))
    for invt in invt_tags:
        funding['Investment Type'].append(invt.text)
    # Scraping data of Amount
    amt_tags = driver.find_elements(By.XPATH,'//table[@id="tablepress-{}"]/tbody/tr/td[9]'.format(i))
    for amt in amt_tags:
        funding['Amount'].append(amt.text)

In [41]:
details=pd.DataFrame(funding)
details.head()

,Date,Startup Name,Industry or Vertical,Sub Vertical,Location,Investor,Investment Type,Amount
0,15/01/2021,Digit Insurance,Financial Services,Insurance Services,Bengaluru,"A91 Partners, Faering Capital, TVS Capital Funds",Venture,"1,80,00,000"
1,28/01/2021,Bombay Shaving Company,Consumer Goods Company,"Shave care, beard care, and skincare products",New Delhi,Reckitt Benckiser,Venture,"6,172,258.50"
2,19/01/2021,DeHaat,AgriTech Startup,online marketplace for farm products and services,Patna,Prosus Ventures,Series C,"30,000,000"
3,19/01/2021,Darwinbox,SaaS,HR Tech,Mumbai,Salesforce Ventures,Seed,"15,000,000"
4,18/01/2021,mfine,Health Tech Startup,AI-powered telemedicine mobile app,Bengaluru,Heritas Capital Management,Venture Round,"16,000,000"


In [42]:
driver.close()

7. Write a program to scrap all the available details of best gaming laptops from digit.in.

In [43]:
driver=webdriver.Chrome("chromedriver.exe")
driver.maximize_window()

driver.get("https://www.digit.in/")

In [44]:
driver.find_element(By.XPATH,"//div[@class='listing_container']//ul//li[9]").click()
time.sleep(3)

In [45]:
#Creating empty lists
Laptop_Name = []
Operating_sys = []
Display = []
Processor = []
Memory = []
Weight = []
Dimensions = []
Graph_proc = []
Price = []
# Scraping the data of laptop names
laptop_name= driver.find_elements(By.XPATH,"//div[@class='right-container']/div/a/h3")
for name in laptop_name:
    Laptop_Name.append(name.text)
    
    
#Scraping the data of operating system
try:
    op_sys = driver.find_elements(By.XPATH,"//div[@class='product-detail']/div/ul/li[1]/div/div")
    for os in op_sys:
        Operating_sys.append(os.text)
except NoSuchElementException:
    pass


#Scraping data of display of the laptop
try:
    display= driver.find_elements(By.XPATH,"//div[@class='product-detail']/div/ul/li[2]/div/div")
    for disp in display:
        Display.append(disp.text)
except NoSuchElementException:
    pass


#Scraping data of Processor
try:
    processor = driver.find_elements(By.XPATH,"//div[@class='Spcs-details'][1]/table/tbody/tr[5]/td[3]")
    for pro in processor:
        Processor.append(pro.text)
except NoSuchElementException:
    pass

#Scraping data of memory
try:
    memory = driver.find_elements(By.XPATH,"//div[@class='Spcs-details'][1]/table/tbody/tr[6]/td[3]")
    for memo in memory:
        Memory.append(memo.text)
except NoSuchElementException:
    pass



#Scraping data of dimensions
try:
    dimension = driver.find_elements(By.XPATH,"//div[@class='Spcs-details'][1]/table/tbody/tr[8]/td[3]")
    for dim in dimension:
        Dimensions.append(dim.text)
except NoSuchElementException:
    pass


#Scraping data of Graph processor
try:
    graph = driver.find_elements(By.XPATH,"//div[@class='Spcs-details'][1]/table/tbody/tr[7]/td[3]")
    for gra in graph:
        Graph_proc.append(gra.text)
except NoSuchElementException:
    pass


#Scraping data of price
try:
    price = driver.find_elements(By.XPATH,"//td[@class='smprice']")
    for pri in price:
        Price.append(pri.text.replace('₹','Rs '))
except NoSuchElementException:
    pass

In [46]:
print(len(Laptop_Name),len(Operating_sys),len(Display),len(Processor),len(Memory),len(Dimensions),len(Graph_proc),len(Price))

10 10 10 10 10 10 10 10


In [47]:
Gaming_Laptop=pd.DataFrame({})
Gaming_Laptop['Laptop Name'] = Laptop_Name
Gaming_Laptop['Operating system'] = Operating_sys
Gaming_Laptop['Display'] = Display
Gaming_Laptop['Processor'] = Processor
Gaming_Laptop['Memory'] = Memory
Gaming_Laptop['Dimensions'] = Dimensions
Gaming_Laptop['Graphical Processor'] = Graph_proc
Gaming_Laptop['Price'] = Price
Gaming_Laptop

,Laptop Name,Operating system,Display,Processor,Memory,Dimensions,Graphical Processor,Price
0,MSI TITAN GT77-12UHS,WINDOWS 11 HOME OS,"17.3"" (3840 X 2160) DISPLAY",Windows 11 Home,64 GB DDR5 RAM & 2 TB SSD,397 x 330 x 23 mm dimension & 3.3 kg weight,16 GB DDR6 NVIDIA GeForce RTX 3080 Ti Graphics...,N/A
1,ALIENWARE X17 R2,WINDOWS 11 HOME OS,"17.3"" (1920X1080) DISPLAY",Windows 11 Home,32 GB DDR5 RAM & 1 TB SSD,399.23 x 299.57 x 14.75 mm dimension & 3.02 kg...,"16 GB DDR6 NVIDIA GeForce RTX 3080 Ti, Graphic...","Rs 389,990"
2,ACER PREDATOR TRITON 500 SE PT516-52S,WINDOWS 11 HOME OS,"16"" (2560 X 1600) DISPLAY",Windows 11 Home,32 GB DDR5 RAM & 2 TB SSD,358 x 262 x 19.9 mm dimension & 2.4 kg weight,8 GB DDR6 NVIDIA GeForce RTXTM 3070 Ti Graphic...,"Rs 300,000"
3,OMEN BY HP (16-B1371TX),WINDOWS 11 HOME OS,"16.1"" (2560 X 1440) DISPLAY",Windows 11 Home,8 GB DDR5 RAM & 1 TB SSD,369 x 248 x 23 mm dimension & 2.32 kg weight,8 GB GDDR6 NVIDIA GeForce RTX 3070 Graphics card,"Rs 194,000"
4,ACER PREDATOR HELIOS 300 AN515-45 (NH.QBRSI.0,WINDOWS 11 HOME OS,"15.6"" (2560 X 1440) DISPLAY",Windows 11 Home,16 GB DDR4 RAM & 512 GB SSD,363 x 255 x 23.9 mm dimension & 2.4 kg weight,8 GB DDR6 NVIDIA GeForce RTX 3070 Graphics card,"Rs 172,999"
5,MSI DELTA 15 (A5EFK-083IN),WINDOWS 11 HOME OS,"15.6"" (1920 X 1080) DISPLAY",Windows 11 Home,16 GB DDR4 RAM & 1 TB SSD,357 x 247 x 19 mm dimension & 1.9 kg weight,10 GB DDR6 AMD Radeon RX 6700M Graphics card,"Rs 188,990"
6,OMEN BY HP (16-C0141AX),WINDOWS 11 HOME OS,"16.1"" (2560 X 1440) DISPLAY",Windows 11 Home,16 GB DDR4 RAM & 1 TB NVMe,36.92 x 24.8 x 2.3 mm dimension & 2.3 kg weight,8 GB GDDR6 AMD Radeon™ RX 6600M Graphics card,"Rs 129,899"
7,LENOVO LEGION 5I PRO (82RF00MGIN),WINDOWS 11 HOME OS,"16"" (2560 X 1600) DISPLAY",Windows 11 Home,16 GB DDR5 RAM & 1 TB SSD,359.9 x 264.4 x 19.9 mm dimension & 2.5 kg weight,6 GB DDR6 NVIDIA GeForce RTX 3060 Graphics card,"Rs 230,890"
8,ALIENWARE M15 R5 RYZEN EDITION ICC-C780001WIN,WINDOWS 11 HOME OS,"15.6"" (1920 X 1080) DISPLAY",Windows 11 Home,16 GB DDR4 RAM & 512 GB SSD,356.2 x 272.5 x 22.85 mm dimension & 2.69 kg w...,6 GB DDR6 NVIDIA GeForce RTX 3060 Graphics card,"Rs 144,990"
9,LENOVO SLIM 7 GEN 6 (82K8002JIN),WINDOWS 11 HOME OS,15.6 MP | NA DISPLAY,Windows 11 Home,16 GB DDR4 RAM & 1 TB SSD,356 x 252 x 16 mm dimension & 1.9 kg weight,6 GB DDR6 NVIDIA GeForce 3060 Max-Q Graphics card,"Rs 131,990"


In [48]:
driver.close()

8. Write a python program to scrape the details for all billionaires from www.forbes.com. Details to be 
scrapped: “Rank”, “Name”, “Net worth”, “Age”, “Citizenship”, “Source”, “Industry”.


In [53]:
# Connecting to webdriver
driver = webdriver.Chrome("chromedriver.exe")
driver.maximize_window()

# opening the url
driver.get('https://www.forbes.com/')

# clicking on options
driver.find_element(By.XPATH,'/html/body/div[1]/header/nav').click()

In [59]:
#clickng on All Billionaires option 

billionaires=driver.find_element(By.XPATH,'//div[@class="mpBfVZz3"]')
billionaires.click()

all_billionaires=driver.find_element(By.XPATH,'/html/body/div[1]/header/nav/div[1]/div/div/div[2]/ul/li[1]/div[2]/div[2]/ul/li[1]/a')
all_billionaires.click()

In [60]:
# Scrapping required details:

#creating empty lists:
Rank = []
Name = []
Net_Worth = []
Age = []
Citizenship = []
Source = []
Industry = []

# Scraping Ranks

try:
    R = driver.find_elements(By.XPATH,'//div[@class="rank"]')
    for i in R:
        Rank.append(i.text)
except NoSuchElementException:
    Rank.append("-")

#Scraping Names
try:
    N = driver.find_elements(By.XPATH,'//div[@class="personName"]')
    for i in N:
        Name.append(i.text)
except NoSuchElementException:
    Name.append("-")

#Scraping Net Worth
try:
    NW = driver.find_elements(By.XPATH,'//div[@class="netWorth"]')
    for i in NW:
        Net_Worth.append(i.text)
except NoSuchElementException:
    Net_Worth.append("-")
    
#Scraping Age
try:
    A = driver.find_elements(By.XPATH,'//div[@class="age"]')
    for i in A:
        Age.append(i.text)
except NoSuchElementException:
    Age.append("-")

#Scraping country Of Citizenship
try:
    C = driver.find_elements(By.XPATH,'//div[@class="countryOfCitizenship"]')
    for i in C:
        Citizenship.append(i.text)
except NoSuchElementException:
    Citizenship.append("-")
    
#Scraping Source
try:
    S = driver.find_elements(By.XPATH,'//div[@class="source"]')
    for i in S:
        Source.append(i.text)
except NoSuchElementException:
    Source.append("-")

#Scraping Industry
try:
    I = driver.find_elements(By.XPATH,'//div[@class="category"]')
    for i in I:
        Industry.append(i.text)
except NoSuchElementException:
    Industry.append("-")

In [61]:
print(len(Rank),len(Name),len(Net_Worth),len(Age),len(Citizenship),len(Source),len(Industry))

200 200 200 200 200 200 200


In [62]:
df = pd.DataFrame({'Rank':Rank,
                   'Name':Name,
                   'Net_Worth':Net_Worth,
                   'Age':Age,
                   'Citizenship':Citizenship,
                   'Source':Source,
                   'Industry':Industry})
df

,Rank,Name,Net_Worth,Age,Citizenship,Source,Industry
0,1.,Elon Musk,$219 B,,United States,"Tesla, SpaceX",
1,2.,Jeff Bezos,$171 B,,United States,Amazon,
2,3.,Bernard Arnault & family,$158 B,,France,LVMH,
3,4.,Bill Gates,$129 B,,United States,Microsoft,
4,5.,Warren Buffett,$118 B,,United States,Berkshire Hathaway,
...,...,...,...,...,...,...,...
195,192.,Marcel Herrmann Telles,$10.3 B,,Brazil,beer,
196,197.,Leon Black,$10 B,,United States,private equity,
197,197.,Joe Gebbia,$10 B,,United States,Airbnb,
198,197.,David Geffen,$10 B,,United States,"movies, record labels",


In [63]:
driver.close()

9. Write a program to extract at least 500 Comments, Comment upvote and time when comment was posted 
from any YouTube Video.

In [21]:
# Connecting to webdriver
driver = webdriver.Chrome("chromedriver.exe")
driver.maximize_window()

# opening the url
driver.get('https://www.youtube.com/')


In [22]:
# Clicking on any Random Video
driver.find_element(By.XPATH,'/html/body/ytd-app/div[1]/ytd-page-manager/ytd-browse/ytd-two-column-browse-results-renderer/div[1]/ytd-rich-grid-renderer/div[6]/ytd-rich-grid-row[4]/div/ytd-rich-item-renderer[3]/div/ytd-rich-grid-media/div[1]/div[2]/div[1]/h3/a/yt-formatted-string').click()

In [23]:
# Pausing the video
driver.find_element(By.XPATH,'/html/body/ytd-app/div/ytd-page-manager/ytd-watch-flexy/div[5]/div[1]/div/div[1]').click()

In [61]:
# Scrapping required details:

#Creating Empty Lists
Comment = []
Upvote = []
Time = []

# Scraping Comments 
C = driver.find_elements(By.XPATH,'//div[@class="style-scope ytd-expander"]')
for i in C:
    Comment.append(i.text)
    
#Scrapping Upvotes
try:
    U = driver.find_elements(By.XPATH,'//span[@id="vote-count-middle"]')
    for i in U:
        Upvote.append(i.text)
except NoSuchElementException:
    Upvote.append("-")

#Scrapping when comment was posted 
try:
    T = driver.find_elements(By.XPATH,'//a[@class="yt-simple-endpoint style-scope yt-formatted-string"]')
    for i in T:
        Time.append(i.text)
except NoSuchElementException:
    Time.append("-")

In [62]:
print(len(Comment),len(Upvote),len(Time))

522 520 622


In [63]:
df = pd.DataFrame({'Comments':Comment[:500],
                   'Comment upvote':Upvote[:500],
                   'time when comment was posted':Time[:500]})

df

,Comments,Comment upvote,time when comment was posted
0,,339,
1,,4.7K,
2,Also watch :-\nVijay Mallya Scam - https://www...,3.4K,
3,"He was not a scammer , he was just too genius.",1.7K,
4,"I won't say I'm a hero ,let me be what I am, a...",3.3K,
...,...,...,...
495,Even today I think that he was the hero not vi...,,1 year ago
496,My big salute to this real gujrati beacuse he ...,,1 year ago
497,"Tx to Harshad Mehta ,He is great, Intelligent ...",,5 months ago
498,He was great person according to my opinion .....,,1 year ago


In [64]:
driver.close()

10. Write a python program to scrape a data for all available Hostels from https://www.hostelworld.com/ in 
“London” location. You have to scrape hostel name, distance from city centre, ratings, total reviews, 
overall reviews, privates from price, dorms from price, facilities and property description

In [12]:
# Connecting to webdriver
driver = webdriver.Chrome("chromedriver.exe")
driver.maximize_window()

# opening the url
driver.get('https://www.hostelworld.com/')

In [14]:
# Enter the location
loc = driver.find_element(By.XPATH,'/html/body/div[3]/div/div/div[2]/div[1]/div/div/div[4]/div/div[2]/div/div[1]/div/div/div/input')
loc.clear()
loc.send_keys("London")

In [15]:
#choosing frist option
driver.find_element(By.XPATH,'/html/body/div[3]/div/div/div[2]/div[1]/div/div/div[4]/div/div[2]/div/div[1]/div/div/ul/li[2]/div').click()

In [16]:
# Clicking on Let's go!
driver.find_element(By.XPATH,'/html/body/div[3]/div/div/div[2]/div[1]/div/div/div[4]/div/div[2]/div/div[5]/button').click()

In [17]:
# Scrapping required details: 
#scraping hostel name, distance from city centre, ratings, total reviews, overall reviews, 
#privates from price, dorms from price, facilities and property description.

#Creating Empty Lists
Name = []
Distance = []
Rating = []
Reviews = []
Overall = []
Privates = []
Dorms = []
Facilities = []
Property = []


# Scraping Names 
N = driver.find_elements(By.XPATH,'//h2[@class="title title-6"]')
for i in N:
    Name.append(i.text)
    
#Scrapping distance from city centre
try:
    D = driver.find_elements(By.XPATH,'//span[@class="description"]')
    for i in D:
        Distance.append(i.text)
except NoSuchElementException:
    Distance.append("-")

#Scrapping ratings
try:
    RT = driver.find_elements(By.XPATH,'//div[@class="rating rating-summary-container big"]')
    for i in RT:
        Rating.append(i.text[:][0:3])
except NoSuchElementException:
    Rating.append("-")
    
#Scrapping total reviews
try:
    RV = driver.find_elements(By.XPATH,'//div[@class="reviews"]')
    for i in RV:
        Reviews.append(i.text)
except NoSuchElementException:
    Reviews.append("-")
    
#Scrapping overall reviews
try:
    O = driver.find_elements(By.XPATH,'//div[@class="keyword"]')
    for i in O:
        Overall.append(i.text)
except NoSuchElementException:
    Overall.append("-")
    
#Scrapping privates from price
P = driver.find_elements(By.XPATH,'//div[@class="price-col"]')[0::2]
for i in P:
    Privates.append(i.text.replace('\n',': ').replace('No Privates Available','NaN'))
       
#Scrapping Dorms from price
P = driver.find_elements(By.XPATH,'//div[@class="price-col"]')[1::2]
for i in P:
    Dorms.append(i.text.replace('\n',': ').replace('No Dorms Available','NaN'))
    
#Scrapping Facilities
try:
    F = driver.find_elements(By.XPATH,'//div[@class="facilities-label facilities"]')
    for i in F:
        Facilities.append(i.text.replace('\n',', '))
except NoSuchElementException:
    Facilities.append("-")
    
#Scrapping property description.
try:
    PD = driver.find_elements(By.XPATH,'//div[@class="rating-factors prop-card-tablet rating-factors small"]')
    for i in PD:
        Property.append(i.text.replace('\n',', '))
except NoSuchElementException:
    Property.append("-")

In [18]:
print(len(Name),len(Distance),len(Rating),len(Reviews),len(Privates),len(Dorms),len(Facilities),len(Property))

30 30 30 30 30 30 30 30


In [19]:
#scraping hostel name, distance from city centre, ratings, total reviews, overall reviews, 
#privates from price, dorms from price, facilities and property description.
# Creating Data Frame 
df = pd.DataFrame({'Hostel name':Name,
                   'Distance from city centre':Distance,
                   'Rating':Rating,
                   'Total reviews':Reviews,
                   'Privates from price':Privates,
                   'Dorms from price':Dorms,
                   'Facilities':Facilities,
                   'Property description':Property})
df

,Hostel name,Distance from city centre,Rating,Total reviews,Privates from price,Dorms from price,Facilities,Property description
0,Palmers Lodge - Swiss Cottage,Hostel - 6.5km from city centre,8.7,15563 Total Reviews,NaN,Dorms From: Rs2159,"Free WiFi, Follows Covid-19 sanitation guidance","Perfect Location, Superb Staff, Fantastic Clea..."
1,St Christopher's Village,Hostel - 1.8km from city centre,8.2,11475 Total Reviews,Privates From: Rs12372.64 Rs11135,Dorms From: Rs2251.81 Rs2027,"Free WiFi, Follows Covid-19 sanitation guidance","Perfect Location, Superb Staff, Excellent Clea..."
2,Selina Camden,Hostel - 5.5km from city centre,8.6,36 Total Reviews,Privates From: Rs22230 Rs18896,NaN,Free WiFi,"Perfect Location, Superb Staff, Fantastic Clea..."
3,Wombat's City Hostel London,Hostel - 3.6km from city centre,9.0,14008 Total Reviews,Privates From: Rs17676.87 Rs14672,Dorms From: Rs4047.92 Rs3036,"Free WiFi, Follows Covid-19 sanitation guidance","Perfect Location, Superb Staff, Fantastic Clea..."
4,Generator London,Hostel - 3km from city centre,7.8,7137 Total Reviews,Privates From: Rs9698,Dorms From: Rs2619,"Free WiFi, Follows Covid-19 sanitation guidance","Perfect Location, Marvellous Staff, Excellent ..."
5,Astor Museum Inn,Hostel - 2.2km from city centre,8.2,9045 Total Reviews,Privates From: Rs10189,Dorms From: Rs2779,"Free WiFi, Follows Covid-19 sanitation guidance","Perfect Location, Superb Staff, Excellent Clea..."
6,Urbany Hostel London,Hostel - 5.4km from city centre,9.3,453 Total Reviews,Privates From: Rs11004,Dorms From: Rs2017,"Free WiFi, Follows Covid-19 sanitation guidance","Perfect Location, Superb Staff, Fantastic Clea..."
7,Astor Hyde Park,Hostel - 4.3km from city centre,8.5,11665 Total Reviews,NaN,Dorms From: Rs2223,"Free WiFi, Follows Covid-19 sanitation guidance","Perfect Location, Superb Staff, Excellent Clea..."
8,Safestay London Elephant & Castle,Hostel - 1.7km from city centre,7.0,4473 Total Reviews,Privates From: Rs15005,Dorms From: Rs3918,"Free WiFi, Follows Covid-19 sanitation guidance","Wonderful Location, Brilliant Staff, Awesome C..."
9,Smart Camden Inn Hostel,Hostel - 4.4km from city centre,8.4,2919 Total Reviews,NaN,Dorms From: Rs2031,"Free WiFi, Follows Covid-19 sanitation guidance","Perfect Location, Superb Staff, Fantastic Clea..."


In [20]:
driver.close()